In [1]:
from multiprocessing import Pool, TimeoutError
import time
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import math
import itertools
import pickle
import datetime
from numba import jit, cuda, float32, float64

In [2]:
def set_initial_guess(xdata, k):
    np.random.seed(1234)
# set pi mu sigma
    pis = np.random.random(k)
    pis *= 0.9
    pis += 0.1
    pis /= np.sum(pis)
# mu
    mus = np.zeros(k)
# sigma
    sigmas = np.random.random(k)
    sigmas *= 1.5
    sigmas += 0.25
    sigmas *= np.std(xdata)
    return pis, mus, sigmas

In [3]:
df = pd.read_csv('RI.IMOEX_180323_180424_5min.csv', sep=';')
# drop last from every day
df = df.groupby('<DATE>').apply(lambda x: x.iloc[:-2] if len(x) > 1 else x).reset_index(drop=True)

In [4]:
data = []
prevopen = 0
for index, row in df.iterrows():
    if prevopen != 0:
        adata = row[4] - prevopen
        data = np.append(data, adata)
    prevopen = row[4]

In [5]:
text_file = open("d://tmp//base_2.txt", "r")
lines = text_file.readlines()

In [6]:
a = []
for i in lines:
    a.append(float(i))

In [7]:
data = a

In [8]:
data = data[0:1040]

In [9]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [18]:
# init
df = pd.read_csv('RI.IMOEX_180323_180424_5min.csv', sep=';')
    # drop last from every day
df = df.groupby('<DATE>').apply(lambda x: x.iloc[:-2] if len(x) > 1 else x).reset_index(drop=True)

# prepare data
datas = preparedatas()

178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311


In [16]:
def preparedatas():
    winlen = 312
    datas = []
    #for x in range(len(df) - winlen):
    for x in range(178, 312):
        print(x)
        xf = df[x:x+winlen]
        data = np.array([])
        prevopen = 0
        for index, row in xf.iterrows():
            if prevopen != 0:
                adata = row[4] - prevopen
                #            if (abs(adata)< 30):
                data = np.append(data, adata)
            prevopen = row[4]
        #    data = np.append(data,df.iloc[-1][7]-prevopen)
        datas.append(data)
    return datas


In [20]:
min(datas[23])

-14.059999999999945

In [22]:
np.exp(-1 5**2/2.0)

1.3863432936411706e-49

In [10]:
print(data)

[-8.230e+00  3.300e-01 -7.600e-01  4.030e+00 -1.350e+00 -2.780e+00
 -2.930e+00  2.590e+00  6.500e-01  9.460e+00  3.800e-01  2.370e+00
  1.640e+00 -4.040e+00  1.360e+00  2.850e+00 -2.910e+00 -8.000e-02
  1.580e+00  5.800e-01 -2.250e+00 -3.100e+00 -6.700e-01 -7.500e-01
  2.310e+00 -3.660e+00 -1.980e+00  3.240e+00 -2.120e+00 -6.400e-01
  8.700e-01 -8.800e-01 -2.230e+00  2.200e+00 -1.530e+00  1.400e-01
 -1.860e+00  2.350e+00  2.200e+00  4.600e-01 -2.000e-01  7.000e-01
  1.640e+00  2.400e+00 -1.300e-01  1.140e+00  1.100e+00 -8.000e-02
  8.800e-01  1.030e+00  1.140e+00 -9.000e-02 -2.500e-01 -2.700e-01
 -1.400e+00 -8.200e-01  1.600e-01  9.000e-01  1.010e+00  2.300e-01
  5.690e+00 -9.100e-01 -1.600e+00  3.100e-01 -2.000e-01 -7.600e-01
  9.600e-01  3.030e+00 -8.600e-01 -5.000e-01 -1.400e+00  1.230e+00
  9.600e-01  3.000e-01 -8.300e-01  1.460e+00 -2.600e-01  1.710e+00
  4.000e-01 -4.080e+00 -1.100e-01 -2.700e-01  1.230e+00 -2.600e-01
  1.640e+00 -2.480e+00 -1.670e+00 -8.900e-01 -1.670e+00  2.900

In [10]:
k = 10

In [11]:
pis = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
mus = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
sigmas = [0.5, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.8, 3.0]

In [12]:
tol = 0.01
max_iterations = 2
n = len(data)
# ем алг
ll_old = 0
ll_new = 0
for i in range(max_iterations):
    print('{3} [{0}:{1}]EM iteration {2}'.format(iter, os.getpid(), i, str(datetime.datetime.now())))
    pis, mus, sigmas = set_initial_guess(data, k)
# E step
    ws = np.zeros((k, n))
    for j in range(len(mus)):
        for i in range(n):
            ws[j, i] = pis[j] * sts.norm(mus[j], sigmas[j]).pdf(data[i])
    ws /= ws.sum(0)

# M-step
    pis = np.zeros(k)
    for j in range(len(mus)):
        for i in range(n):
            pis[j] += ws[j, i]
    pis /= n

    mus = np.zeros(k)
    for j in range(k):
        for i in range(n):
            mus[j] += ws[j, i] * data[i]
        mus[j] /= ws[j, :].sum()

    sigmas = np.zeros(k)
    for j in range(k):
        for i in range(n):
            sigmas[j] += ws[j, i] * ((data[i] - mus[j]) ** 2)
        sigmas[j] /= ws[j, :].sum()
        sigmas[j] = math.sqrt(sigmas[j])

    print('pis {0}'.format(pis))
    print( 'mus {0}'.format(mus))
    print( 'sigmas {0}'.format(sigmas))

    #
    # update complete log likelihoood
    ll_new = 0.0
    for i in range(n):
        s = 0
        for j in range(k):
            s += pis[j] * sts.norm(mus[j], sigmas[j]).pdf(data[i])
        ll_new += np.log(s)

    if np.abs(ll_new - ll_old) < tol:
        break
    # print 'll = {0}\n'.format(ll_new)
    ll_old = ll_new

2019-05-03 19:30:34.988358 [<built-in function iter>:12844]EM iteration 0
pis [0.04628683 0.10520102 0.0729331  0.11785439 0.13554533 0.05361471
 0.05555699 0.15007071 0.13786118 0.12507573]
mus [ 0.07363021  0.04728053  0.00891124  0.00376498  0.07205053  0.03402416
  0.04682327  0.01507309 -0.00673703 -0.08702928]
sigmas [1.23747533 1.50756625 1.88585143 1.96415632 1.26125159 1.62213229
 1.51149328 0.50244646 2.1658563  2.87240044]
2019-05-03 19:30:51.917360 [<built-in function iter>:12844]EM iteration 1
pis [0.04628683 0.10520102 0.0729331  0.11785439 0.13554533 0.05361471
 0.05555699 0.15007071 0.13786118 0.12507573]
mus [ 0.07363021  0.04728053  0.00891124  0.00376498  0.07205053  0.03402416
  0.04682327  0.01507309 -0.00673703 -0.08702928]
sigmas [1.23747533 1.50756625 1.88585143 1.96415632 1.26125159 1.62213229
 1.51149328 0.50244646 2.1658563  2.87240044]


In [18]:
ws[1]

array([0.00136423, 0.08941365, 0.11118894, ..., 0.08903831, 0.08731396,
       0.02198288])

In [21]:
math.sqrt(2*math.pi)

2.5066282746310002

In [ ]:
def worker_task(iter, zdata, k, pis, mus, sigmas):
    print("{2} start worker {0}:{1}\n".format(iter, os.getpid(), str(datetime.datetime.now())))
    pis, mus, sigmas = set_initial_guess(zdata, k)
    ll, pi, mu, sigma = em_alg_decomposition(iter, zdata, k, pis, mus, sigmas)
    print("{2} end wkr {0}:{1}\n".format(iter, os.getpid(), str(datetime.datetime.now())))
    return [ll, pi, mu, sigma]


def func_star(a_b):
    """Convert `f([1,2])` to `f(1,2)` call."""
    return worker_task(*a_b)

@jit
def preparedatas():

    return datas


# main
if __name__ == '__main__':
    slswindowlen = 7
    pool = Pool(processes=8)  # start 8 worker processes

# init


# prepare data
    datas = preparedatas()

# start
    k = 10

# set initial guess
    pis, mus, sigmas = set_initial_guess(datas[0], k)
    result = pool.map(func_star, zip(range(len(datas)), datas, itertools.repeat(k), itertools.repeat(pis), itertools.repeat(mus), itertools.repeat(sigmas)))

    print(result)
    serialized = pickle.dump(result, open("Output_180323_180424_5min.txt", "wb"))
